# Detecting heatwaves in Australia from 1981-2018

## Import modules and define functions 

In [1]:
from datetime import date 
import xarray as xr
import pandas as pd
import numpy as np
import dask as da
import glob 
import scipy.ndimage as ndimage
from scipy.ndimage.measurements import label, find_objects
#import dask_image.ndmeasure <-- Would make computation faster - but not finished yet
from dask.distributed import LocalCluster, Client
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
local_dir = "/g/data/e14/cp3790/dask-workers"
cluster = LocalCluster(processes=False, local_dir=local_dir)
client = Client(cluster)
client


/g/data3/hh5/public/apps/miniconda3/envs/analysis3-19.10/lib/python3.6/site-packages/distributed/worker.py:485: UserWarning: The local_dir keyword has moved to local_directory
  warnings.warn("The local_dir keyword has moved to local_directory")


Client Scheduler: inproc://10.0.64.18/10543/1 Dashboard: http://10.0.64.18/10543/1:8787/status,Cluster Workers: 1 Cores: 8 Memory: 33.67 GB


In [45]:
def consec(temps):
    labels, _ = label(temps) # labels the occurrence of 1s and gives it an 'event' number 
    slices = find_objects(labels) 
    new_temps = np.zeros(len(temps))
    
    for i in slices:
        if temps[i].size >= 3:
            new_temps[i] = new_temps[i].size
        else:
            new_temps[i] = 0
    return new_temps

In [46]:
def consec_ufunc(array, axis):
    result = np.apply_along_axis(
        consec, 
        axis,
        array
    )
    return result 

In [47]:
def consecutive(dataarray):
    result = xr.apply_ufunc(
        consec_ufunc, dataarray,
        input_core_dims=[['time']],
        output_core_dims=[['time']],
        kwargs={'axis':-1},
        output_sizes={'time':len(dataarray['time'])}
    )
    return result.transpose(*dataarray.dims)

## Opening files

In [29]:
files = sorted(glob.glob('/g/data/e14/cp3790/Charuni/ERA5-new/era5_dailytmax_*.nc'))

obs_aus = xr.open_mfdataset(files, combine='by_coords').sel(time=slice('1981', '2018'), longitude=slice(113, 154), latitude=slice(-10, -44))

obs_aus_tmax = obs_aus["dmax"].load() #.sel(time=~((era5_dailytmax_aus["dmax"].time.dt.month == 2) & (era5_dailytmax_aus["dmax"].time.dt.day == 29)))
obs_aus_tmax.attrs['units'] = 'deg C'

In [55]:
# Opens threshold file that has been created previously 

threshold = xr.open_dataarray('/g/data/e14/cp3790/Charuni/threshold-australia.nc')

## Code

### Identifying consecutive heatwave days from 1981-2018 using the SciPy function 

In [38]:
aus_heatwave_days = (obs_aus_tmax.groupby('time.dayofyear') > threshold).astype(int)

In [39]:
aus_heatwave_days

<xarray.DataArray (time: 13879, latitude: 137, longitude: 165)>
array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 0],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]],

       ...,

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 0, 1],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]],

       [[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        ...,
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1],
        [0, 0, 0, ..., 1, 1, 1]]])
Coordinates:
  * longitude  (longitude) float32 113.0 113.25 113.5 ... 153.5 153.75 154.0
  * latitude   (latitude) float32 -10.0 -10.25 -10.5 ... -43.5 -43.75 -44.0
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2018-12-31
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365

In [56]:
aus_heatwave_events = consecutive(aus_heatwave_days)

In [57]:
aus_heatwaves_1981_2018 = aus_heatwave_events.to_dataset(name='duration')

In [58]:
aus_heatwaves_1981_2018

<xarray.Dataset>
Dimensions:    (latitude: 137, longitude: 165, time: 13879)
Coordinates:
  * longitude  (longitude) float32 113.0 113.25 113.5 ... 153.5 153.75 154.0
  * latitude   (latitude) float32 -10.0 -10.25 -10.5 ... -43.5 -43.75 -44.0
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2018-12-31
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
Data variables:
    duration   (time, latitude, longitude) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0

In [59]:
aus_heatwaves_1981_2018.to_netcdf('/g/data/e14/cp3790/Charuni/aus_heatwaves_1981_2018.nc')